In [ ]:
#Import necessary modules
import requests
from bs4 import BeautifulSoup
import pandas as pd

print("All modules imported successfully!")

All modules imported successfully!


In [2]:
#Saving the URL of the website to scrape
url = "https://books.toscrape.com/"

#Sending a GET request to the website
response = requests.get(url)

#Checking if the request was successful
if response.status_code == 200:
    print("Successfully accessed the website!")
else:
    print(f"Failed to access the website. Status code: {response.status_code}")

Successfully accessed the website!


In [ ]:
soup = BeautifulSoup(response.text, 'html.parser')

In [5]:
soup.title

<title>
    All products | Books to Scrape - Sandbox
</title>

In [ ]:
#Process to get all the books on a page
books = soup.find_all('article', class_='product_pod')
print(type(books))
print(len(books))
books[0]

<class 'bs4.element.ResultSet'>
20


<article class="product_pod">
<div class="image_container">
<a href="catalogue/a-light-in-the-attic_1000/index.html"><img alt="A Light in the Attic" class="thumbnail" src="media/cache/2c/da/2cdad67c44b002e7ead0cc35693c0e8b.jpg"/></a>
</div>
<p class="star-rating Three">
<i class="icon-star"></i>
<i class="icon-star"></i>
<i class="icon-star"></i>
<i class="icon-star"></i>
<i class="icon-star"></i>
</p>
<h3><a href="catalogue/a-light-in-the-attic_1000/index.html" title="A Light in the Attic">A Light in the ...</a></h3>
<div class="product_price">
<p class="price_color">Â£51.77</p>
<p class="instock availability">
<i class="icon-ok"></i>
    
        In stock
    
</p>
<form>
<button class="btn btn-primary btn-block" data-loading-text="Adding..." type="submit">Add to basket</button>
</form>
</div>
</article>

In [14]:
#Looping through all the pages to extract all books
base_url = "https://books.toscrape.com/catalogue/page-{}.html"

all_books = []

for page in range(1, 51):
    url = base_url.format(page)
    response = requests.get(url)
    soup = BeautifulSoup(response.text, 'html.parser')

    books = soup.find_all('article', class_='product_pod')
    all_books.extend(books)

print(f"Total number of books: {len(all_books)}")

Total number of books: 1000


In [ ]:
#Extracting the info from each book
titles = []
prices = []
ratings = []
availability = []

for book in all_books:
    title = book.h3.a['title']
    titles.append(title)

    